In [2]:
import pandas as pd
import pm4py
from tqdm import tqdm 
from datetime import datetime, timedelta

In [7]:
application_log = pm4py.read_xes(r"C:\Users\boris\OneDrive\Bureaublad\Datascience and Entrepreneurship\Year 2\Semester 1\Real-Time process mining\Assignment 2\BPI Challenge 2017.xes.gz")
offer_log = pm4py.read_xes(r"C:\Users\boris\OneDrive\Bureaublad\Datascience and Entrepreneurship\Year 2\Semester 1\Real-Time process mining\Assignment 2\BPI Challenge 2017 - Offer log.xes.gz")

df_application = pm4py.convert_to_dataframe(application_log)
df_offer = pm4py.convert_to_dataframe(offer_log)

df_application.to_csv('app_logs.csv')
df_offer.to_csv('offer_logs.csv')

Exception: File does not exist

In [8]:
df_application = pd.read_csv(r'C:\Users\boris\OneDrive\Bureaublad\Datascience and Entrepreneurship\Year 2\Semester 1\Real-Time process mining\app_logs.csv')
df_offer = pd.read_csv(r'C:\Users\boris\OneDrive\Bureaublad\Datascience and Entrepreneurship\Year 2\Semester 1\Real-Time process mining\offer_logs.csv')

In [9]:
df_offer = df_offer.sample(frac=0.05, random_state=1)
df_application = df_application.sample(frac=0.05, random_state = 1)

In [10]:
declined_ids = list(df_application.loc[df_application['concept:name'] == 'A_Denied']['case:concept:name'].unique())
accepted_ids = list(df_application.loc[df_application['concept:name'] == 'A_Pending']['case:concept:name'].unique())
cancelled_ids = list(df_application.loc[df_application['concept:name'] == 'A_Cancelled']['case:concept:name'].unique())

In [11]:
# 31,509 loan applications in total
# 98 noise 
len(declined_ids)+ len(accepted_ids)+len(cancelled_ids)

1583

In [12]:
len(declined_ids), len(accepted_ids), len(cancelled_ids)

(170, 892, 521)

In [13]:
def get_max_time(ids, i):
    return max(df_application.loc[df_application['case:concept:name'] == ids[i]]['time:timestamp'])

In [14]:
timestamp_list = []
for i in tqdm(range(len(accepted_ids))):
    timestamp_list.append(get_max_time(accepted_ids, i))
#all(timestamp_list[i] <= timestamp_list[i+1] for i in range(len(timestamp_list) - 1))

100%|██████████| 892/892 [00:05<00:00, 153.55it/s]


In [342]:
accepted_ids_w_time = sorted([(id, time) for id, time in zip(accepted_ids, timestamp_list)], key=lambda x: x[1])

ids_length = len(accepted_ids_w_time)
train_num = int(ids_length * 0.8)

train_ids = [pair[0] for pair in accepted_ids_w_time[:train_num]]
test_ids = [pair[0] for pair in accepted_ids_w_time[train_num:]]

train_df = df_application.loc[df_application['case:concept:name'].isin(train_ids)]
test_df = df_application.loc[df_application['case:concept:name'].isin(test_ids)]

train_event_log = pm4py.convert_to_event_log(train_df)
pm4py.write_xes(train_event_log, 'approved_train.xes')

test_event_log = pm4py.convert_to_event_log(test_df)
pm4py.write_xes(test_event_log, 'approved_test.xes')

exporting log, completed traces ::   0%|          | 0/713 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/179 [00:00<?, ?it/s]

# Part 1

In [12]:
def create_train_test(df_application, ids):

    timestamp_list = []
    for i in tqdm(range(len(ids))):
        timestamp_list.append(get_max_time(ids, i))
    ids_w_time = sorted([(id, time) for id, time in zip(ids, timestamp_list)], key=lambda x: x[1])

    ids_length = len(ids_w_time)
    train_num = int(ids_length * 0.8)

    train_ids = ids_w_time[:train_num]
    test_ids = ids_w_time[train_num:]

    train_df = df_application.loc[df_application['case:concept:name'].isin(train_ids)]
    test_df = df_application.loc[df_application['case:concept:name'].isin(test_ids)]

    return train_df, test_df
    

In [13]:
def save_xes(train_df, test_df, pre):

    train_event_log = pm4py.convert_to_event_log(train_df)
    pm4py.write_xes(train_event_log, f'{pre}_train.xes')

    test_event_log = pm4py.convert_to_event_log(test_df)
    pm4py.write_xes(test_event_log, f'{pre}_test.xes')

In [14]:
cancelled_tr, cancelled_te = create_train_test(df_application, cancelled_ids)

100%|██████████| 521/521 [00:06<00:00, 77.01it/s] 


In [15]:
save_xes(cancelled_tr, cancelled_te, 'cancelled')

exporting log, completed traces :: : 0it [00:00, ?it/s]

exporting log, completed traces :: : 0it [00:00, ?it/s]

In [16]:
approved_tr, approved_te = create_train_test(df_application, accepted_ids)
save_xes(approved_tr, approved_te, 'approved')

100%|██████████| 892/892 [00:10<00:00, 83.91it/s] 


exporting log, completed traces :: : 0it [00:00, ?it/s]

exporting log, completed traces :: : 0it [00:00, ?it/s]

# Part 2 building dataset

In [17]:
def aggregate_df(df):
    """ 
    Aggregate the df of current events in the case

    Output: 
        result -> could be a pandas series
    """

    # record the timestamp of the last activity
    result = df.iloc[0]

    return result


In [18]:
def add_to_aggregate(result, df_row):
    """ 
    When a new event happens, add the event info to the current aggregated result.

    Input: 
         result: the current aggregated result
         df_row: pandas df row representing the new event
    Output:
        result: the new aggregated result
    """

    # record the timestamp of the last activity
    result = df_row.iloc[0]

    return result

In [20]:
def create_prefix_part2(df_application, app_ids, end_event, start_event='A_Accepted'):
    
    app_id_list = list(df_application['case:concept:name'].unique())

    # TODO:
    # create a return df
    return_df = pd.DataFrame()

    # extracting prefix for each application
    for app_id in app_id_list:
        
        events_app = df_application.loc[df_application['case:concept:name'] == app_id]
        events_app.reset_index(drop=True, inplace=True)

        cur_id = starting_row_id = events_app.loc[events_app['concept:name'] == 'A_Accepted'].index[0]
        pre_events = events_app.iloc[:starting_row_id]
        # TODO: 
        # aggregate events_app from row 0 to starting_row_id
        result = aggregate_df(pre_events)
        
        ending_row_id = events_app.loc[events_app['concept:name'] == end_event].index[0]
        cur_id += 1
        
        while cur_id < ending_row_id:
            new_row = events_app.iloc[cur_id]
            # TODO: 
            # add new event row info to the aggregated result
            result = add_to_aggregate(pre_events)

            # Update the return_df -> add new row
            # target y: end_event

            cur_id += 1

        return return_df


# Part 3 building dataset


In [10]:
#build function to retrieve minimum time 
def get_min_time(ids, i):
    """define function to retrieve the minimum time"""
    return min(df_application.loc[df_application["case:concept:name"] == ids[i]]["time:timestamp"])

#get the minimum time for the accepted ids
min_timestamp_list = []
for i in tqdm(range(len(accepted_ids))):
    min_timestamp_list.append(get_min_time(accepted_ids, i))

#create a list with every id and the start time, and another list with every id and the end time
accepted_ids_begin = sorted([(id, time) for id, time in zip(accepted_ids, min_timestamp_list)], key=lambda x: x[1])
accepted_ids_end = sorted([(id, time) for id, time in zip(accepted_ids, timestamp_list)], key=lambda x: x[1])

#generate dataframe for begin and end times 
df_accepted_ids_time_begin = pd.DataFrame(accepted_ids_begin, columns = ["case:concept:name", "begin"])
df_accepted_ids_time_end = pd.DataFrame(accepted_ids_end, columns = ["case:concept:name", "end"])

#merge dataframes on case:concept:name
df_accepted_timestamps = df_accepted_ids_time_begin.merge(df_accepted_ids_time_end, on = "case:concept:name")

#keep relevant time formatting
df_accepted_timestamps["begin"] = df_accepted_timestamps["begin"].map(lambda x: str(x)[:19])
df_accepted_timestamps["end"] = df_accepted_timestamps["end"].map(lambda x: str(x)[:19])

#create function to calculate the difference in time from a dataframe with two columns containing dates and time
def calc_duration(end, begin):
    """calculate the difference in time using datetime.strptime"""
    return (datetime.strptime(end, "%Y-%m-%d %H:%M:%S") - datetime.strptime(begin, "%Y-%m-%d %H:%M:%S")).total_seconds()

#empty list to gather differences
duration = []

#retrieve the difference between begin and end of the trace and add to a list 
for i in range(0, len(df_accepted_timestamps)):
    duration.append(calc_duration(df_accepted_timestamps.iloc[i]["end"], df_accepted_timestamps.iloc[i]["begin"]))

#add the time difference to the df 
df_accepted_timestamps["duration"] = duration

#remove all cases with case time duration 0 
df_accepted_timestamps = df_accepted_timestamps.loc[df_accepted_timestamps["duration"] > 0]

#get indexes to filter outliers top and bottom 5%
outliers_index = list(range(0, round(0.05 * len(df_accepted_timestamps)))) + list(range(round(0.95 * len(df_accepted_timestamps)), len(df_accepted_timestamps)))

#sort values from small to big time difference and drop respective rows
df_accepted_timestamps = df_accepted_timestamps.sort_values(by= "duration", ignore_index = True).drop(labels = outliers_index, axis = "index")

100%|██████████| 892/892 [00:12<00:00, 73.43it/s]


In [87]:
df_accepted_timestamps_begin = df_accepted_timestamps.sort_values(by = "begin").reset_index(drop = True)
df_accepted_timestamps_end = df_accepted_timestamps.sort_values(by = "end").reset_index(drop = True)

#test set range
begin_index_test = round(0.8 * len(df_accepted_timestamps_begin))
begin_time_test = df_accepted_timestamps_begin.iloc[begin_index_test]["begin"]
x = calc_duration(max(df_accepted_timestamps["end"]), begin_time_test)

#train set range
end_index_train = round(0.8 * len(df_accepted_timestamps_end))
end_time_train = df_accepted_timestamps_end.iloc[end_index_train]["end"]
y = calc_duration(end_time_train, min(df_accepted_timestamps["begin"]))
 
total_with_overlap = x + y
total_time = calc_duration(max(df_accepted_timestamps["end"]), min(df_accepted_timestamps["begin"]))
overlap_span = total_with_overlap - total_time

overlap_train = 0.8 * overlap_span
overlap_test = 0.2 * overlap_span

end_time_train_with_overlap = datetime.strptime(end_time_train, "%Y-%m-%d %H:%M:%S") 
date_index_train = datetime.strftime((end_time_train_datetime - timedelta(seconds = overlap_train)), "%Y-%m-%d %H:%M:%S")

begin_time_test_with_overlap = datetime.strptime(begin_time_test, "%Y-%m-%d %H:%M:%S") 
date_index_test = datetime.strftime((begin_time_test_datetime + timedelta(seconds = overlap_test)), "%Y-%m-%d %H:%M:%S")

df_train = df_accepted_timestamps_end.loc[df_accepted_timestamps_end["end"] < date_index_train]
df_test = df_accepted_timestamps_begin.loc[df_accepted_timestamps_begin["begin"] > date_index_test]

2016-11-10 08:31:15 2016-10-28 20:54:15 2016-10-25 18:00:01 2016-10-28 20:54:15


# Part 4 building dataset

In [25]:
new_df = df_application.loc[df_application['case:concept:name'] == 'Application_428409768']

In [ ]:
new_df.reset_index(drop=True, inplace=True)

In [ ]:
new_df

In [ ]:
# def create_prefix_part2(df_application, app_ids, end_event, start_event='A_Accepted'):
def create_dataset_part4(df_application, app_ids, current_event, target_event):
    """
    Encode original applications 
    """
    
    app_id_list = list(df_application['case:concept:name'].unique())

    # TODO: get all unique activities before target_event
    # TODO: get all unique resources before target_event
    # use abrove info to aggregate below

    for app_id in app_id_list:

        all_events = df_application.loc[df_application['case:concept:name'] == app_id]

        # aggregate rows of events and append the timestamp of the target_event

        # Encode each case attribute as a feature (or one-hot encode categorical case attributes)

        # For each numerical event attribute, apply an aggregation function (e.g. average) over the sequence of values taken by this attribute in the prefix
        # sum up time as one feature?

        # For each categorical event attribute, encode each possible value of that attribute as a numerical feature. 
        
        
        